# Example Model Servers with Seldon

Follow [docs](https://docs.seldon.io/projects/seldon-core/en/latest/) to install Seldon Core.

In [1]:
!kubectl create namespace seldon || echo "Already exists"

Error from server (AlreadyExists): namespaces "seldon" already exists
Already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [3]:
import json

## Serve SKLearn Iris Model

**Note**: Seldon has adopted the industry-standard Open Inference Protocol (OIP) and is no longer maintaining the Seldon and TensorFlow protocols. This transition allows for greater interoperability among various model serving runtimes, such as MLServer. To learn more about implementing OIP for model serving in Seldon Core 1, see [MLServer](https://docs.seldon.ai/mlserver).

We strongly encourage you to adopt the OIP, which provides seamless integration across diverse model serving runtimes, supports the development of versatile client and benchmarking tools, and ensures a high-performance, consistent, and unified inference experience.

In order to deploy SKLearn artifacts, we can leverage the [pre-packaged SKLearn inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html).
The exposed API can follow either:
- Open Inference Protocol or V2 Protocol.
- Seldon protocol. 




### Default Seldon protocol

To deploy and start serving an SKLearn artifact using Seldon's default protocol, we can use a config like the one below:

In [4]:
%%writefile ../servers/sklearnserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  predictors:
  - graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/v1.19.0-dev/sklearn/iris
    name: default
    replicas: 1
    svcOrchSpec:
      env:
      - name: SELDON_LOG_LEVEL
        value: DEBUG

Overwriting ../servers/sklearnserver/samples/iris.yaml


We can then apply it to deploy it to our Kubernetes cluster.

In [5]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [6]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed we can send our sklearn model requests

#### REST Requests

In [7]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8004/seldon/seldon/sklearn/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}, 'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.19.0-dev'}}}


In [8]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="sklearn", namespace="seldon")

2025-11-14 15:49:55.724159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763135395.741442  346478 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763135395.746701  346478 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-14 15:49:55.764768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
r = sc.predict(gateway="istio", transport="rest", shape=(1, 4))
print(r)
assert r.success == True

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.8999394626768058
    values: 0.8994698493108374
    values: 0.42215210127769531
    values: 0.18248998704464248
  }
}

Response:
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.5477171093892841, 0.39867534283487427, 0.0536075477758417]}}, 'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.19.0-dev'}}}


#### gRPC Requests

In [10]:
r = sc.predict(gateway="istio", transport="grpc", shape=(1, 4))
print(r)
assert r.success == True

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.1990521763963784, 0.3312814313703717, 0.9923164124382778, 0.1493811594722928]}}}
Response:
{'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.19.0-dev'}}, 'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.1088934676232064, 0.46084338952890413, 0.4302631428478895]}}}


In [11]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:sklearn -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8004 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.19.0-dev'}}, 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}}


And delete the model we deployed

In [12]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


### Open Inference Protocol or V2 Protocol

For example, we can consider the config below:

In [13]:
%%writefile ./resources/iris-sklearn-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  protocol: v2
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris-0.23.2/lr_model
      name: classifier
    name: default
    replicas: 1

Overwriting ./resources/iris-sklearn-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [14]:
!kubectl apply -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [15]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [17]:
import json

import requests

inference_request = {
    "inputs": [
        {"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}
    ]
}

endpoint = "http://localhost:8004/seldon/seldon/sklearn/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "631543ed-ec76-44fa-80da-282559bca6f6",
  "parameters": {},
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1,
        1
      ],
      "datatype": "INT64",
      "parameters": {
        "content_type": "np"
      },
      "data": [
        2
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [18]:
!kubectl delete -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In order to deploy XGBoost models, we can leverage the [pre-packaged XGBoost inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The V2 protocol

### Default Seldon protocol

We can deploy a XGBoost model uploaded to an object store by using the XGBoost model server implementation as shown in the config below:

In [22]:
%%writefile resources/iris-xgboost.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Writing resources/iris-xgboost.yaml


And then we apply it to deploy it to our kubernetes cluster

In [23]:
!kubectl apply -f resources/iris-xgboost.yaml

seldondeployment.machinelearning.seldon.io/xgboost unchanged


In [24]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

deployment "xgboost-default-0-classifier" successfully rolled out


#### Rest Requests

In [25]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8004/seldon/seldon/xgboost/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [2.0]}, 'meta': {'requestPath': {'classifier': 'seldonio/xgboostserver:1.19.0-dev'}}}


In [26]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="xgboost", namespace="seldon")

In [27]:
r = sc.predict(gateway="istio", transport="rest", shape=(1, 4))
print(r)
assert r.success == True

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.95307866192819213
    values: 0.31049339777734397
    values: 0.13684652902137184
    values: 0.092120926929723956
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [0.0]}}, 'meta': {'requestPath': {'classifier': 'seldonio/xgboostserver:1.19.0-dev'}}}


#### gRPC Requests

In [28]:
r = sc.predict(gateway="istio", transport="grpc", shape=(1, 4))
print(r)
assert r.success == True

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.6444000470434165, 0.9512148944684768, 0.9480745682068712, 0.6717077131691793]}}}
Response:
{'meta': {'requestPath': {'classifier': 'seldonio/xgboostserver:1.19.0-dev'}}, 'data': {'tensor': {'shape': [1], 'values': [0.0]}}}


In [29]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:xgboost -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8004 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {'requestPath': {'classifier': 'seldonio/xgboostserver:1.19.0-dev'}}, 'data': {'ndarray': [2]}}


And delete the model we deployed

In [30]:
!kubectl delete -f resources/iris-xgboost.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


### V2 protocol

We can deploy a XGBoost model, exposing an API compatible with v2 protocol by specifying the `protocol` of our `SeldonDeployment` as `v2`.
For example, we can consider the config below:

In [31]:
%%writefile ./resources/iris-xgboost-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost-v2
spec:
  name: iris
  protocol: v2
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: iris
    name: default
    replicas: 1

Writing ./resources/iris-xgboost-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [32]:
!kubectl apply -f ./resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io/xgboost-v2 created


In [33]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost-v2 -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "xgboost-v2-default-0-iris" rollout to finish: 0 of 1 updated replicas are available...
deployment "xgboost-v2-default-0-iris" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [34]:
import json

import requests

inference_request = {
    "inputs": [
        {"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}
    ]
}

endpoint = "http://localhost:8004/seldon/seldon/xgboost-v2/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "iris",
  "model_version": "v0.1.0",
  "id": "24c5228a-b237-4b60-af55-ffe71700106c",
  "parameters": {},
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1,
        1
      ],
      "datatype": "FP32",
      "parameters": {
        "content_type": "np"
      },
      "data": [
        2.0
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [35]:
!kubectl delete -f ./resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io "xgboost-v2" deleted


## Serve Tensorflow MNIST Model
We can deploy a tensorflow model uploaded to an object store by using the
tensorflow model server implementation as the config below.

This notebook contains two examples, one which shows how you can use the
TFServing prepackaged serve with the Seldon Protocol, and a second one which
shows how you can deploy it using the tensorlfow protocol (so you can send
requests of the exact format as you would to a tfserving server).

### Serve Tensorflow MNIST Model with Seldon Protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Seldon protocol.

In [36]:
%%writefile ./resources/mnist_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
        - name: model_input
          type: STRING
          value: images
        - name: model_output
          type: STRING
          value: scores
    name: default
    replicas: 1

Writing ./resources/mnist_rest.yaml


In [37]:
!kubectl apply -f ./resources/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [38]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "tfserving-default-0-mnist-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "tfserving-default-0-mnist-model" successfully rolled out


In [39]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="tfserving", namespace="seldon")

#### REST Request

In [40]:
r = sc.predict(gateway="istio", transport="rest", shape=(1, 784))
print(r)
assert r.success == True

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.73408911087987849
    values: 0.059386437513020973
    values: 0.80258888548962
    values: 0.29918038125358515
    values: 0.3081984948310651
    values: 0.65781368274450813
    values: 0.42116834519764024
    values: 0.15746159261096393
    values: 0.8015853851572553
    values: 0.76778110313037884
    values: 0.43732641492059732
    values: 0.033427722208757116
    values: 0.15087789247206318
    values: 0.516658327862452
    values: 0.956961119332205
    values: 0.90283330388905536
    values: 0.53954703415573424
    values: 0.51401468096395675
    values: 0.17394749083090444
    values: 0.597688914595066
    values: 0.78916870009970741
    values: 0.028914345415039344
    values: 0.32198404552080528
    values: 0.22547701557170108
    values: 0.050210657867989728
    values: 0.51718441209896482
    values: 0.98731593471098844
    values: 0.99299313918733623
    values: 0.28174857688

#### gRPC Request

In [41]:
r = sc.predict(gateway="istio", transport="grpc", shape=(1, 784))
print(r)
assert r.success == True

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 784], 'values': [0.038977502354621496, 0.8092387777572863, 0.5060296526720705, 0.7505576633985839, 0.8622858546834442, 0.8322268981163985, 0.5601405638435004, 0.10624241219393227, 0.1847018068501487, 0.735387138159413, 0.8308685843725097, 0.7578922912460482, 0.06043461893456736, 0.36148046600970507, 0.3927269891402978, 0.44127752103775597, 0.9531143891569789, 0.5248346466404331, 0.4162726800091423, 0.3009767265376647, 0.8373962314880555, 0.35346022750709405, 0.13598751657675578, 0.043938601942853484, 0.45853370120824366, 0.6479546301621568, 0.44042095504294465, 0.35220038752194893, 0.6807374451993506, 0.9238807589806531, 0.9983679433354998, 0.5448495685241334, 0.9160489758855478, 0.793400277707031, 0.3282637076929378, 0.5988848684016196, 0.06795358994557343, 0.5154973167998476, 0.34665464866279116, 0.04922711452669648, 0.019974825334150514, 0.5088941774716137, 0.6278307296166731, 0.7450653344766891, 0.79459304

And delete the model we deployed

In [42]:
!kubectl delete -f ./resources/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


### Serve Tensorflow Model with Tensorflow protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Tensorflow protocol.

In [43]:
%%writefile ./resources/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      name:  halfplustwo
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Writing ./resources/halfplustwo_rest.yaml


In [44]:
!kubectl apply -f ./resources/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io/hpt created


In [45]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=hpt -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "hpt-default-0-halfplustwo" rollout to finish: 0 of 1 updated replicas are available...
deployment "hpt-default-0-halfplustwo" successfully rolled out


In [46]:
import json
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8004/seldon/seldon/hpt/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

{'predictions': [2.5, 3.0, 4.5]}


In [47]:
X=!cd ../executor/proto && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:hpt -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8004 tensorflow.serving.PredictionService/Predict
d=json.loads("".join(X))
print(d)
assert(d["outputs"]["x"]["floatVal"][0] == 2.5)

{'outputs': {'x': {'dtype': 'DT_FLOAT', 'tensorShape': {'dim': [{'size': '3'}]}, 'floatVal': [2.5, 3, 3.5]}}, 'modelSpec': {'name': 'halfplustwo', 'version': '123', 'signatureName': 'serving_default'}}


In [48]:
!kubectl delete -f ./resources/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io "hpt" deleted


## Serve MLFlow Elasticnet Wines Model

In order to deploy MLflow models, we can leverage the [pre-packaged MLflow inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/mlflow.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The V2 protocol.

### Default Seldon protocol

We can deploy an MLFlow model uploaded to an object store by using the MLFlow model server implementation as the config below:

In [49]:
%%writefile ./resources/elasticnet_wine.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - componentSpecs:
    - spec:
        # We are setting high failureThreshold as installing conda dependencies
        # can take long time and we want to avoid k8s killing the container prematurely
        containers:
        - name: classifier
          livenessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
    graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/v1.18.0/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1

Writing ./resources/elasticnet_wine.yaml


In [50]:
!kubectl apply -f ./resources/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [51]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "mlflow-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
^C


#### REST requests

In [48]:
X=!curl -s -d '{"data": {"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
   -X POST http://localhost:8004/seldon/seldon/mlflow/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [5.275558760255375]}, 'meta': {'requestPath': {'classifier': 'seldonio/mlflowserver:1.18.0'}}}


In [49]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="mlflow", namespace="seldon")

In [50]:
r = sc.predict(gateway="istio", transport="rest", shape=(1, 11))
print(r)
assert r.success == True

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 11
    values: 0.5653756616147143
    values: 0.8420723980045103
    values: 0.5062962018951862
    values: 0.5278571532571091
    values: 0.8664864216170011
    values: 0.45973619467487525
    values: 0.9263453669947618
    values: 0.6150362295808135
    values: 0.5089439556920178
    values: 0.9282257119213217
    values: 0.7953848591020709
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [5.244154959197477]}}, 'meta': {'requestPath': {'classifier': 'seldonio/mlflowserver:1.18.0'}}}


#### gRPC Requests

In [51]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
         -rpc-header seldon:mlflow -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8004 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {'requestPath': {'classifier': 'seldonio/mlflowserver:1.18.0'}}, 'data': {'ndarray': [5.275558760255375]}}


In [52]:
r = sc.predict(gateway="istio", transport="grpc", shape=(1, 11))
print(r)
assert r.success == True

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 11], 'values': [0.49331471179570063, 0.6224359102414585, 0.9804140644783785, 0.3380112388366434, 0.8942524358731484, 0.2837043142995588, 0.9867737754039229, 0.47015800689221765, 0.5398691845908773, 0.6674452878134565, 0.4007242201724862]}}}
Response:
{'meta': {'requestPath': {'classifier': 'seldonio/mlflowserver:1.18.0'}}, 'data': {'tensor': {'shape': [1], 'values': [5.204512993729161]}}}


In [52]:
!kubectl delete -f ./resources/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted


### V2 protocol

We can deploy a MLflow model, exposing an API compatible with v2 protocol by specifying the `protocol` of our `SeldonDeployment` as `v2`.
For example, we can consider the config below:

In [54]:
%%writefile ./resources/elasticnet_wine_v2.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  protocol: v2  # Activate v2 protocol
  name: wines
  predictors:
    - graph:
        children: []
        implementation: MLFLOW_SERVER
        modelUri: gs://seldon-models/v1.12.0-dev/mlflow/elasticnet_wine
        name: classifier
      name: default
      replicas: 1

Overwriting ./resources/elasticnet_wine_v2.yaml


In [55]:
!kubectl apply -f ./resources/elasticnet_wine_v2.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [56]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "mlflow-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "mlflow-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [57]:
import json

import requests

inference_request = {
    "parameters": {"content_type": "pd"},
    "inputs": [
        {
            "name": "fixed acidity",
            "shape": [1],
            "datatype": "FP32",
            "data": [7.4],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "volatile acidity",
            "shape": [1],
            "datatype": "FP32",
            "data": [0.7000],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "citric acidity",
            "shape": [1],
            "datatype": "FP32",
            "data": [0],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "residual sugar",
            "shape": [1],
            "datatype": "FP32",
            "data": [1.9],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "chlorides",
            "shape": [1],
            "datatype": "FP32",
            "data": [0.076],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "free sulfur dioxide",
            "shape": [1],
            "datatype": "FP32",
            "data": [11],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "total sulfur dioxide",
            "shape": [1],
            "datatype": "FP32",
            "data": [34],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "density",
            "shape": [1],
            "datatype": "FP32",
            "data": [0.9978],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "pH",
            "shape": [1],
            "datatype": "FP32",
            "data": [3.51],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "sulphates",
            "shape": [1],
            "datatype": "FP32",
            "data": [0.56],
            "parameters": {"content_type": "np"},
        },
        {
            "name": "alcohol",
            "shape": [1],
            "datatype": "FP32",
            "data": [9.4],
            "parameters": {"content_type": "np"},
        },
    ],
}

endpoint = "http://localhost:8004/seldon/seldon/mlflow/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "732f2602-cc8a-479d-86c8-ee99bafd1a1e",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP64",
      "parameters": null,
      "data": [
        6.016145744177844
      ]
    }
  ]
}


In [58]:
!kubectl delete -f ./resources/elasticnet_wine_v2.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
